In [ ]:
import pickle
import pandas as pd
import numpy as np
import detectda as dtda
import matplotlib.pyplot as plt
from skimage import filters
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import sys
from joblib import Parallel, delayed
from scipy.stats import mode
sys.path.append('..')

In [ ]:
import bclr

In [ ]:
vids2 = np.load("Experiment2_Plus1_Images1000.npy")
vid201 = vids2[201]
vid902 = vids2[902]

In [ ]:
plt.imshow(filters.gaussian(vids2[27][45], sigma=2), cmap='gray')

In [ ]:
file=open('experiment2_data.pickle', 'rb')
exp2=pickle.load(file)

In [ ]:
np.mean(np.array([mod.post_k_mode for mod in exp2['bcc_tda']])==40)

In [ ]:
plt.rcParams["font.family"] = "Nunito"

In [ ]:
kcp_tda_all = exp2['kcp_tda']
kcp_pca_all = exp2['kcp_pca']

cf_tda_all = exp2['cf_tda']
cf_pca_all = exp2['cf_pca']

bclr_tda_all = exp2['bcc_tda']
bclr_pca_all = exp2['bcc_pca']

In [ ]:
def rmse(x):
    return np.sqrt(np.mean(x**2))

cp_bclr_mses_tda = [rmse(bclr.post_k-40)  for bclr in bclr_tda_all]
cp_bclr_mses_pca = [rmse(bclr.post_k-40)  for bclr in bclr_pca_all]

In [ ]:
bclr_modes_t = np.array([bclr.post_k_mode for bclr in bclr_tda_all])
bclr_means_t = np.array([np.mean(bclr.post_k) for bclr in bclr_tda_all])

In [ ]:
np.mean(np.abs(bclr_means_t-40))

In [ ]:
print(rmse(bclr_modes_t-40))

print(rmse(bclr_means_t-40))

In [ ]:
print("Mean RMSE: %0.3f" % np.mean(cp_bclr_mses_tda))
print("SE RMSE: %0.3f" % np.std(cp_bclr_mses_tda))

In [ ]:
print("Mean RMSE: %0.3f" % np.mean(cp_bclr_mses_pca))
print("SE RMSE: %0.3f" % np.std(cp_bclr_mses_pca))

In [ ]:
plt.hist(bclr_tda_all[4].k_draws_[2500:], bins=np.arange(1, 51))

In [ ]:
print(kcp_tda_all[27])
print(cf_tda_all[27])

In [ ]:
plt.plot(bclr_tda_all[27].k_draws_)

In [ ]:
hh = pd.DataFrame(bclr_tda_all[0].k_draws_[2500:]).value_counts(normalize=True)

ind = np.where(hh.cumsum() > 1-0.05)[0][0]

In [ ]:
def hpm(arr, alpha=0.05):
    probs = pd.DataFrame(arr).value_counts(normalize=True)
    ind = np.where(probs.cumsum() > 1-alpha)[0][0]
    high_mass = np.array([i[0] for i in probs.index[:(ind+1)]])
    return high_mass

In [ ]:
bool(np.isin(40, hpm(bclr_tda_all[0].k_draws_[2500:], 0.05)))

In [ ]:
hpm(bclr_tda_all[0].k_draws_[2500:], 0.05)

In [ ]:
hpm(bclr_tda_all[4].k_draws_[2500:])
arr = bclr_tda_all[4].k_draws_[2500:]
probs = pd.DataFrame(arr).value_counts(normalize=True)
probs.cumsum()

In [ ]:
for al in [0.5, 0.2, 0.1, 0.05, 0.01]:    
    m1_bclr_tda = [(np.quantile(bclr.k_draws_[2500:], al/2), np.quantile(bclr.k_draws_[2500:], 1-al/2)) for bclr in bclr_tda_all] 
    m2_bclr_tda = [bool(np.isin(40, hpm(bclr.k_draws_[2500:], al))) for bclr in bclr_tda_all]
                  
    print(1-al, np.mean([x[1] >= 40 and x[0] <= 40 for x in m1_bclr_tda]), "tda quantile")
    print(1-al, np.mean(m2_bclr_tda), "tda hpm")              

In [ ]:
m2_bclr_tda

What about Highest Posterior Density credible set? 

### bclr changepoint detection

In [ ]:
rands= np.random.choice(np.arange(0, 1000), 9, replace=False)

In [ ]:
k_draws_after_burn = [bclr_tda_all[i].k_draws_[2500:] for i in range(1000)]

In [ ]:
kdab = np.array(k_draws_after_burn).flatten()

In [ ]:
plt.hist(kdab, bins=np.arange(1, 51, 1))

In [ ]:
cp_bclr_tda_mean = [np.mean(bclr.k_draws_[2500:], keepdims=False) for bclr in bclr_tda_all]

In [ ]:
cp_bclr_tda = [mode(bclr.k_draws_[2500:], keepdims=False).mode for bclr in bclr_tda_all]
cp_bclr_pca = [mode(bclr.k_draws_[2500:], keepdims=False).mode for bclr in bclr_pca_all]

In [ ]:
cp_coverage = [(np.quantile(bclr.k_draws_[2500:], 0.05), np.quantile(bclr.k_draws_[2500:], 1)) for bclr in bclr_tda_all]

In [ ]:
np.mean([i[0] <= 40 and i[1] >= 40 for i in cp_coverage])

In [ ]:
hj = [np.mean(np.array([mode(bclr.k_draws_[2500:], keepdims=False).mode for bclr in bclr_tda_all if bclr.k_draws_[0] == j])==40) for j in range(1,51)]

In [ ]:
plt.bar(x=np.arange(1, 51), height=hj)

In [ ]:
np.mean(np.array(cp_bclr_pca))

In [ ]:
plt.rcParams["font.family"] = "Nunito"

In [ ]:
import seaborn as sns

In [ ]:
sns.set_style("whitegrid")

In [ ]:
print("RMSE for bclr, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_bclr_tda)-40)**2)))
print("RMSE for bclr, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_bclr_pca)-40)**2)))

print("\n")

print("Probability k=40 for bclr, TDA features: %0.3f" % np.mean(np.array(cp_bclr_tda)==40))
print("Probability k=40 for bclr, PCA features: %0.3f" % np.mean(np.array(cp_bclr_pca)==40))

print("Std Err: Probability k=40 for bclr, TDA features: %0.3f" % (np.std(np.array(cp_bclr_tda)==40)/np.sqrt(1000)))
print("Std Err: Probability k=40 for bclr, PCA features: %0.3f" % (np.std(np.array(cp_bclr_pca)==40)/np.sqrt(1000)))

In [ ]:
np.std(np.array(cp_bclr_tda)==40)/np.sqrt(1000)

In [ ]:
print("Probability k=40 for bclr, TDA features (first 50): %0.3f" % np.mean(np.array(cp_bclr_tda[:50])==40))

In [ ]:
for i in range(5, 0, -1):
    print("Probability "+str(40-i)+" <= k <= "+str(40+i)+" for bclr, TDA features: %0.3f" % np.mean(
        np.logical_and(np.array(cp_bclr_tda) <= 40+i, np.array(cp_bclr_tda) >= 40-i)))

In [ ]:
np.mean((np.array(cp_bclr_tda_mean)-40)**2)

Let's look at the beta coefficients

In [ ]:
cp_bclr_tda_beta = np.array([bclr.beta_draws_[2500:] for bclr in bclr_tda_all])
names = ['mean_0_mid', 'variance_0_mid', 'skewness_0_mid', 'kurtosis_0_mid', 'median_0_mid', 'iqr_0_mid', 'q25_0_mid', 'q75_0_mid', 'pers_entr_0_life', 'alps_0_life', 'mean_0_life', 'variance_0_life','skewness_0_life', 'kurtosis_0_life', 'median_0_life', 'iqr_0_life', 'q25_0_life', 'q75_0_life', 'mean_1_mid', 'variance_1_mid', 'skewness_1_mid', 'kurtosis_1_mid', 'median_1_mid', 'iqr_1_mid','q25_1_mid', 'q75_1_mid', 'pers_entr_1_life', 'alps_1_life','mean_1_life', 'variance_1_life', 'skewness_1_life', 'kurtosis_1_life','median_1_life', 'iqr_1_life', 'q25_1_life', 'q75_1_life']
beta_cors = np.array([np.corrcoef(bcbeta, rowvar=False) for bcbeta in cp_bclr_tda_beta])
bcors = np.mean(beta_cors, axis=0)

In [ ]:
nam1 = []
nam2 = []
cors = []
for i in range(35):
    for j in range(i+1, 36):
        nam1.append(names[i])
        nam2.append(names[j])
        cors.append(bcors[i,j])

In [ ]:
cor_df = pd.DataFrame({'Var1': nam1, 'Var2': nam2, 'Corr': cors})
cor_df.reindex(cor_df['Corr'].abs().sort_values(ascending=False).index)[:20]

In [ ]:
beta_snr = np.array([np.mean(bcbeta, axis=0)**2/np.var(bcbeta, axis=0) for bcbeta in cp_bclr_tda_beta])

In [ ]:
abc = pd.Series(np.array(names)[np.argmax(beta_snr, axis=1)]).value_counts(normalize=True)
prop_largest = abc.reindex(names).fillna(0)
df_prop = pd.DataFrame({'names': names, 'mean_snr': np.mean(beta_snr, axis=0), 'sd_snr': np.std(beta_snr, axis=0),
              'prop_highest': prop_largest}).sort_values("prop_highest", ascending=False)

In [ ]:
df_prop

In [ ]:
nam_5 = list(df_prop.head(5).index)
nam_list = np.array(names)

In [ ]:
cor_index = [np.where(nam_list == nam)[0][0] for nam in nam_5]

In [ ]:
for i in cor_index:
    for j in cor_index:
        if i<j:
            print(nam_list[i], "\t\t", nam_list[j], "\t\t", np.round(bcors[i,j],3))

### Changeforest changepoint detection

In [ ]:
cp_cf_tda = [a['best_split'] for a in cf_tda_all]
cp_cf_pca = [a['best_split'] for a in cf_pca_all]

print("RMSE for CF, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_cf_tda)-40)**2)))
print("RMSE for CF, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_cf_pca)-40)**2)))

In [ ]:
print("Probability k=40 for CF, TDA features: %0.3f" % np.mean(np.array(cp_cf_tda)==40))
print("Probability k=40 for CF, PCA features: %0.3f" % np.mean(np.array(cp_cf_pca)==40))

print("Std Err: Probability k=40 for CF, TDA features: %0.3f" % (np.std(np.array(cp_cf_tda)==40)/np.sqrt(1000)))
print("Std Err: Probability k=40 for CF, PCA features: %0.3f" % (np.std(np.array(cp_cf_pca)==40)/np.sqrt(1000)))

In [ ]:
np.min(np.array(cp_cf_tda))

In [ ]:
plt.hist(np.array(cp_cf_tda), bins=20)

Kernel Changepoint detection using <code>ruptures</code> package

In [ ]:
cp_kcp_tda = [a['changepoint'][0] for a in kcp_tda_all]
cp_kcp_pca = [a['changepoint'][0] for a in kcp_pca_all]

Changepoint distribution for Kernel Changepoint detection and dynamic programming using TDA features

In [ ]:
cp_kcp_tda

In [ ]:
print("RMSE for KCP, TDA features: %0.3f" % np.sqrt(np.mean((np.array(cp_kcp_tda)-40)**2)))
print("RMSE for KCP, PCA features: %0.3f" % np.sqrt(np.mean((np.array(cp_kcp_pca)-40)**2)))

In [ ]:
print("Probability k=40 for KCP, TDA features: %0.3f" % np.mean(np.array(cp_kcp_tda)==40))
print("Probability k=40 for KCP, PCA features: %0.3f" % np.mean(np.array(cp_kcp_pca)==40))

print("Std Err: Probability k=40 for KCP, TDA features: %0.3f" % (np.std(np.array(cp_kcp_tda)==40)/np.sqrt(1000)))
print("Std Err: Probability k=40 for KCP, PCA features: %0.3f" % (np.std(np.array(cp_kcp_pca)==40)/np.sqrt(1000)))

In [ ]:
print("Worst deviation from actual KCP: "+str(int(np.max(np.abs(np.array(cp_kcp_tda)-40)))))
print("Worst deviation from actual CF: "+str(int(np.max(np.abs(np.array(cp_cf_tda)-40)))))
print("Worst deviation from actual bclr: "+str(int(np.max(np.abs(np.array(cp_bclr_tda)-40)))))

In [ ]:
plt.hist(np.array(cp_kcp_tda), bins=20)

### How does it compare to the integration methods?

In [ ]:
#289, 208, 318, 582, 956 are good ones...

from scipy.stats import multivariate_normal

#705, 902, 948 yield interesting behavior
ind = np.random.randint(1, 1001, 1)[0]
print(str(ind)+"\n")

#Small variance of coordinates works really well!!!
prior_cov = np.diag(np.repeat(0.25, 36))

#Could the sigma value = 2, be obstructing the behavior here? 
aa_k = bclr.BayesCC_kappa(bclr_tda_all[ind].X, prior_cov = prior_cov, n_iter = 5000, print_res=False)
print(aa_k['probs'].idxmax(), "\n\n", aa_k['probs'].max())

print("\n"+"Changepoint from beta sim method (bclr): "+str(cp_bclr_tda[ind]))

print("\n"+"Changepoint from KCP: "+str(cp_kcp_tda[ind]))

print("\n"+"Changepoint from CF: "+str(cp_cf_tda[ind]))


### Below, we process the data for usage by the methods using R